# Phase-space factor with complex decay masses

In [ ]:
%config InlineBackend.figure_format = 'svg'
import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

rub_blue = "#17365c"
rub_green = "#8dae10"

To avoid a vertical cut between the threshold ($(m_1+m_2)^2$) and the pseudo-threshold ($(m_1-m_2)^2$), define the phase space factor as
$$
\rho(s) = \frac{\sqrt{s-(m_1-m_2)^2}\sqrt{s-(m_1+m_2)^2}}{s}
\,.
$$

In [ ]:
def rho(s, m1, m2):
    return np.sqrt(s - (m1 - m2) ** 2) * np.sqrt(s - (m1 + m2) ** 2) / s

If instead you use
$$
\rho_\text{wrong}(s) =
\frac{\sqrt{\left(s-(m_1-m_2)\right)^2\left(s-(m_1+m_2)^2\right)}}{s}
\,,
$$
you get

In [ ]:
%matplotlib inline


def rho_wrong_cut(s, m1, m2):
    return np.sqrt((s - (m1 - m2) ** 2) * (s - (m1 + m2) ** 2)) / s


s = np.linspace(0, 1.2, num=200) + 1e-8j
ma = 0.13957
mb = 0.493677
plt.rc("font", size=14)
fig, axes = plt.subplots(figsize=(5, 5), nrows=2, sharex=True)
ax1, ax2 = axes
for ax in axes:
    ax.spines["left"].set_position("zero")
    ax.spines["bottom"].set_position("zero")
    ax.spines["right"].set_color("none")
    ax.spines["top"].set_color("none")
    ax.xaxis.set_ticks_position("bottom")
    ax.yaxis.set_ticks_position("left")
    ax.set_ylim(-0.5, 0.8)
    ax.set_xticks([])
    ax.set_yticks([])
ax2.set_xlabel(R"$s$")
ax1.set_ylabel(R"$\rho(s)$")
ax2.set_ylabel(R"$\rho_\mathrm{wrong}(s)$")
ax1.plot(s.real, rho(s, ma, mb).real, color=rub_blue, label=R"$\Re\,\rho(s)$")
ax1.plot(s.real, rho(s, ma, mb).imag, color=rub_green, label=R"$\Im\,\rho(s)$")
ax2.plot(s.real, rho_wrong_cut(s, ma, mb).real, color=rub_blue, label=R"$\Re\,\rho(s)$")
ax2.plot(
    s.real, rho_wrong_cut(s, ma, mb).imag, color=rub_green, label=R"$\Im\,\rho(s)$"
)
ax1.legend()
fig.tight_layout()
plt.show(fig)

We can take complex masses for the decay masses $m_1$ and $m_2$ in order to **mimic unstable decay products**.

In [ ]:
sliders = dict(
    m1_real=w.FloatSlider(description="Re 𝑚₁", value=0.8, min=0, max=2, step=0.01),
    m1_imag=w.FloatSlider(description="Im 𝑚₁", value=0, min=-1, max=+1, step=0.01),
    m2_real=w.FloatSlider(description="Re 𝑚₂", value=1.5, min=0, max=2, step=0.01),
    m2_imag=w.FloatSlider(description="Im 𝑚₂", value=0, min=-1, max=+1, step=0.01),
    z_max=w.FloatSlider(description="𝑧 scale", value=1, min=0.01, max=5, step=0.01),
    y_scale=w.FloatSlider(description="𝑦 scale", value=1, min=0.05, max=2, step=0.05),
)
UI = w.VBox([
    w.HBox([sliders["m1_real"], sliders["m1_imag"]]),
    w.HBox([sliders["m2_real"], sliders["m2_imag"]]),
    w.HBox([sliders["z_max"], sliders["y_scale"]]),
])

In [ ]:
%matplotlib widget
x_min, x_max = 0, 10
y_max = 1
x = np.linspace(x_min, x_max, num=300)
X, Y = np.meshgrid(x, np.linspace(-y_max, +y_max, num=200))
S = X + 1j * Y

plt.rc("font", size=7)
fig, ax = plt.subplots(dpi=200, figsize=(4.2, 2.4), rasterized=True)
fig.canvas.header_visible = False
fig.canvas.toolbar_visible = False
ax.set_xlabel(R"$\Re\,s$")
ax.set_ylabel(R"$\Im\,s$")
ax.set_ylim(-y_max, +y_max)
ax.spines["left"].set_position("zero")
ax.spines["bottom"].set_position("zero")
ax.spines["right"].set_color("none")
ax.spines["top"].set_color("none")
ax.xaxis.set_ticks_position("bottom")
ax.yaxis.set_ticks_position("left")


def update_plot(m1_real, m1_imag, m2_real, m2_imag, z_max, y_scale):
    m1 = complex(m1_real, m1_imag)
    m2 = complex(m2_real, m2_imag)
    Z = rho(S, m1, m2).imag
    thr_min = (m1_real - m2_real) ** 2
    thr_max = (m1_real + m2_real) ** 2
    y = rho(x + 1e-8j, m1, m2)
    y /= max(
        np.nanmax(y[x > thr_min].real),
        np.nanmax(y[x > thr_min].imag),
    )
    y *= 0.98 * y_scale
    global lines, mesh
    if mesh is None or lines is None:
        lines = [
            ax.axvline(thr_min, color="C0", linestyle="dashed", label="$(m_1-m_2)^2$"),
            ax.axvline(thr_max, color="C1", linestyle="dashed", label="$(m_1+m_2)^2$"),
            ax.plot(x, y.real, label=R"$\Re\,\rho(s)$", color=rub_blue)[0],
            ax.plot(x, y.imag, label=R"$\Im\,\rho(s)$", color=rub_green)[0],
        ]
        mesh = ax.pcolormesh(X, Y, Z, cmap="coolwarm", vmin=-z_max, vmax=+z_max)
        fig.colorbar(mesh, ax=ax, label=R"$\Im\,\rho(s)$", pad=0.03)
    else:
        for line, thr in zip(lines[:2], [thr_min, thr_max]):
            line.set_xdata([thr])
        for line, yi in zip(lines[2:], [y.real, y.imag]):
            line.set_ydata(yi)
        mesh.set_array(Z.ravel())
        mesh.set_clim(-z_max, +z_max)
    fig.canvas.draw_idle()


lines = mesh = None
OUTPUT = w.interactive_output(update_plot, controls=sliders)
ax.legend(loc="lower right")
fig.tight_layout()
display(UI, OUTPUT)